# Introductory Data Exploration of Brazilian Cities
In this dataset we have a population projection for each Brazilian city in the year of 2013.



In [1]:
import pandas as pd
import numpy as np


cities = pd.read_excel('../data/Cidades - estimativa 2013.xlsx',
                       dtype={'COD. UF': np.str, 'COD. MUNIC': str},
                       sheetname=None, header=0)

In [2]:
data = pd.DataFrame()
for key in cities.keys():
    data = pd.concat([data, cities[key]])
    
data = data.reset_index(drop=True)
data.shape

(5584, 5)

We should see 5570 rows because that's the number of cities that IBGE says that Brazil have. The different amount of rows leads me to believe there are metadata from the `.xlsx` messing with our data

## Translating column names

In [3]:
data.rename(columns={
        'UF': 'state',
        'COD. UF': 'state_id',
        'COD. MUNIC': 'city_id',
        'NOME DO MUNICÍPIO': 'city_name',
        'POPULAÇÃO ESTIMADA': 'population_projection'
    }, inplace=True)
data.head()

,state,state_id,city_id,city_name,population_projection
0,RO,11,15,Alta Floresta D'Oeste,25728
1,RO,11,379,Alto Alegre dos Parecis,13827
2,RO,11,403,Alto Paraíso,19459
3,RO,11,346,Alvorada D'Oeste,17399
4,RO,11,23,Ariquemes,101269


## Formating `city_id`

Formatting `city_id` to conform with the ids displayed on the Brazilian cesus files

In [4]:
data['city_id'] = data['city_id'].apply(lambda x: x.zfill(5))

## Checking out a `unique_id` for each city

In [5]:
data[data['city_id'] == '00108']

,state,state_id,city_id,city_name,population_projection
1831,BA,29,00108,Abaíra,9132
5583,DF,53,00108,Brasília,2789761


In [6]:
UNIQUE_IDS = data.loc[:,['state_id', 'city_id']]

for i in range(len(UNIQUE_IDS['state_id'])):
    UNIQUE_IDS.loc[i,'ids'] = '{}{}'.format(UNIQUE_IDS.loc[i,'state_id'],
                                              UNIQUE_IDS.loc[i,'city_id'])

UNIQUE_IDS.head()

,state_id,city_id,ids
0,11,00015,1100015
1,11,00379,1100379
2,11,00403,1100403
3,11,00346,1100346
4,11,00023,1100023


In [7]:
len(set(UNIQUE_IDS['ids']))

5570

In [8]:
UNIQUE_IDS.shape

(5584, 3)

In [9]:
brazilian_states = {'RO': 'rondonia',
                    'AC': 'acre',
                    'AM': 'amazonas',
                    'RR': 'roraima',
                    'PA': 'para',
                    'AP': 'amapa',
                    'TO': 'tocantis',
                    'MA': 'maranhao',
                    'PI': 'piaui',
                    'CE': 'ceara',
                    'RN': 'rio_grande_do_norte',
                    'PB': 'paraiba',
                    'PE': 'pernambuco',
                    'AL': 'alagoas',
                    'SE': 'sergipe',
                    'BA': 'bahia',
                    'MG': 'mina_gerais',
                    'ES': 'epirito_santo',
                    'RJ': 'rio_de_janeiro',
                    'SP': 'sao_paulo',
                    'PR': 'parana',
                    'SC': 'santa_catarina', 
                    'RS': 'rio_grande_do_sul',
                    'MS': 'mato_grosso_do_sul',
                    'MT': 'mato_grosso',
                    'GO': 'goias',
                    'DF': 'distrito_federal'}

census_link = "ftp.ibge.gov.br/Censos/Censo_Demografico_2010/resultados/total_populacao_{}.zip"